# Inplementation of Internal State-based Uncertainty Estimation

1. 数据集预处理：将不同格式的数据集处理成input+gt的格式，方便判断模型的correctness，这一部分的采用固定不可调整的prompt，即Context: Question: Options: Answer:格式
2. 生成回复，为每个模型确定一个prompt，一个max_new_tokens数，然后生成回复
3. 计算回复部分的correctness指标，判断模型的回复是否正确
4. 计算uncertainty指标，包括PE, LN-PE, SAR, Ours
5. 计算AUROC，绘制AUROC/Correctness-Threshold曲线

In [1]:
from utils import *

datasets.disable_caching()
torch.set_grad_enabled(False)

# Eval Result Config
model_names = [
    "vicuna-7b-v1.1",
    "vicuna-13b-v1.1",
    "vicuna-33b-v1.3",
]

dst_names = [
    "sciq",
    "coqa",
    "triviaqa",
    "medmcqa",
    "MedQA-USMLE-4-options",
]

c_metrics = [
    'rougel',
    'sentsim',
    'include'
]

dst_types = [
    "short",
    "long",
]

acc_map = {
    "vicuna-7b-v1.1": {
        "sciq": 0.60,
        "coqa": 0.8,
        "triviaqa": 0.55,
        "medmcqa": 0.30,
        "MedQA-USMLE-4-options": 0.30
    },
    "vicuna-13b-v1.1": {
        "sciq": 0.0,
        "coqa": 0.0,
        "triviaqa": 0.0,
        "medmcqa": 0.0,
        "MedQA-USMLE-4-options": 0.0
    },
    "vicuna-33b-v1.3": {
        "sciq": 0.0,
        "coqa": 0.0,
        "triviaqa": 0.0,
        "medmcqa": 0.0,
        "MedQA-USMLE-4-options": 0.0
    }
}

model_names_alias = {
    "vicuna-7b-v1.1": "Vicuna-7B",
    "vicuna-13b-v1.1": "Vicuna-13B",
    "vicuna-33b-v1.3": "Vicuna-33B"
}

dst_names_alias = {
    "sciq": "SciQ",
    "coqa": "CoQA",
    "triviaqa": "TriviaQA",
    "medmcqa": "MedMCQA",
    "MedQA-USMLE-4-options": "MedQA"
}

u_metric_alias = {
    "u_score_pe": "PE",
    "u_score_ln_pe": "LN-PE",
    "u_score_token_sar": "TokenSAR",
    "u_score_sent_sar": "SentSAR",
    "u_score_sar": "SAR",
    "u_score_ls": "LS",
    "u_score_se": "SE",
    "u_score_ours_mean_soft_rougel": "Ours(MSRL)",
    "u_score_ours_last_soft_rougel": "Ours(LSRL)",
    "u_score_ours_mean_soft_include": "Ours(MSIN)",
    "u_score_ours_last_soft_include": "Ours(LSIN)",
    "u_score_ours_mean_soft_sentsim": "Ours(MSSI)",
    "u_score_ours_last_soft_sentsim": "Ours(LSSI)"

}


def get_cached_result_path(model_name, dst_name, dst_type, dst_split):
    return f"cached_results/{model_name}/{dst_type}/{dst_name}_{dst_split}"


def get_eval_main_result_path(model_name, dst_name, dst_type):
    return f"eval_results/{model_name}/{dst_name}_{dst_type}"


def get_eval_cross_result_path(model_name, train_dst_name, train_dst_type, test_dst_name, test_dst_type, c_metric):
    return f"cross_eval_results/{model_name}/{c_metric}/v_c_{train_dst_name}_{train_dst_type}_mean_soft_best.pth/{test_dst_name}_{test_dst_type}"


def get_c_th_by_acc(test_dst, c_metric, acc):
    sorted_c_scores = sorted(list(test_dst[c_metric]), reverse=True)
    c_th = sorted_c_scores[int(len(sorted_c_scores) * acc)]
    return c_th


def show_dst_case(test_dst):
    print(test_dst.column_names)
    for i in range(0, 10):
        for k in ['washed_answer', 'gt', 'options', 'include']:
            print(f"{k}: {test_dst[i][k]}")
        print()


def get_acc_by_c_th(test_dst, c_metric, c_th):
    return sum([1 if s > c_th else 0 for s in test_dst[c_metric]]) / len(test_dst)

/mnt/petrelfs/guoyiqiu/miniconda3/envs/mi/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# Merge Train Dataset
model_name = 'vicuna-7b-v1.1'
train_size_per_dataset = 2000
val_size_per_dataset = 100


def merge_dst(model_name, dst_names, dst_types, dst_split, data_size):
    all_dst = [Dataset.load_from_disk(get_cached_result_path(model_name, dst_name, dst_type, dst_split)) for dst_name in dst_names for dst_type in dst_types]

    all_columns = ['dst_template', 'question', 'input', 'gt', 'options', 'answer', 'output']

    def fill_missing_columns(dst: Dataset):
        for column in dst.column_names:
            if column not in all_columns:
                dst = dst.remove_columns(column)
        if 'options' not in dst.column_names:
            dst = dst.add_column('options', [['#####'] for i in range(len(dst))])
        return dst

    all_dst = [dst if len(dst) <= data_size else dst.select(range(data_size)) for dst in all_dst]
    all_dst = [fill_missing_columns(dst) for dst in all_dst]
    merged_dst = datasets.concatenate_datasets(all_dst)
    print(len(merged_dst))
    save_path = get_cached_result_path(model_name, 'all', 'merged', dst_split)
    os.makedirs(save_path, exist_ok=True)
    merged_dst.save_to_disk(save_path)
    print(f"Save Merged Dataset to {save_path}")


# merge_dst(model_name, dst_names, dst_types, 'train', train_size_per_dataset)
# merge_dst(model_name, dst_names, dst_types, 'validation', val_size_per_dataset)

20000


Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Save Merged Dataset to cached_results/vicuna-7b-v1.1/merged/all_train
1000


Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Save Merged Dataset to cached_results/vicuna-7b-v1.1/merged/all_validation


In [45]:
# Main Results: Get Main Results

c_metric = 'include'
c_th = 0.3

model_names_index = [model_names_alias[name] for name in model_names for i in range(len(dst_names))]
dst_names_index = [dst_names_alias[name] for name in dst_names] * len(model_names)
multi_index = pd.MultiIndex.from_tuples(zip(model_names_index, dst_names_index), names=['Model', 'Dataset'])
columns = ['ACC'] + [alias for u_metric, alias in u_metric_alias.items() if (c_metric in u_metric) or 'ours' not in u_metric]
main_results_short = pd.DataFrame(columns=columns, index=multi_index).astype(float)
main_results_long = deepcopy(main_results_short)

for model_name in model_names:
    for dst_name in dst_names:
        for dst_type in dst_types:
            new_row = {k: 0. for k in main_results_short.columns}
            result_path = get_eval_main_result_path(model_name, dst_name, dst_type)
            if os.path.exists(result_path):
                test_dst = Dataset.load_from_disk(result_path)
                new_row['ACC'] = get_acc_by_c_th(test_dst, c_metric, c_th) * 100
                for u_metric, u_metric_name in u_metric_alias.items():
                    if u_metric_name in columns and u_metric in test_dst.column_names:
                        new_row[u_metric_name] = get_auroc(test_dst, u_metric, c_metric, c_th) * 100
            result = main_results_short if dst_type == 'short' else main_results_long
            result.loc[(model_names_alias[model_name], dst_names_alias[dst_name])] = new_row

print(f"Correctness Metric: {c_metric} Threshold: {c_th} AUROC Results")
print("Short Prompt Main Result:")
display(main_results_short)

print("Long Prompt Main Result:")
display(main_results_long)

# print(main_results_short.to_latex(index=True, float_format="%.2f"))
# print(main_results_long.to_latex(index=True, float_format="%.2f"))

Correctness Metric: include Threshold: 0.3 AUROC Results
Short Prompt Main Result:


ACC         PE      LN-PE   TokenSAR    SentSAR  \
Model      Dataset                                                      
Vicuna-7B  SciQ      64.9  29.391262  41.410191  44.585797  30.682751   
           CoQA      68.2  35.083642  47.118169  48.082775  33.491949   
           TriviaQA  36.3  25.849908  27.537181  27.638811  25.951970   
           MedMCQA   19.0  36.008122  48.321637  50.280052  35.152697   
           MedQA     22.8  46.921303  49.607422  49.439256  44.972616   
Vicuna-13B SciQ       0.0   0.000000   0.000000   0.000000   0.000000   
           CoQA       0.0   0.000000   0.000000   0.000000   0.000000   
           TriviaQA   0.0   0.000000   0.000000   0.000000   0.000000   
           MedMCQA    0.0   0.000000   0.000000   0.000000   0.000000   
           MedQA      0.0   0.000000   0.000000   0.000000   0.000000   
Vicuna-33B SciQ       0.0   0.000000   0.000000   0.000000   0.000000   
           CoQA       0.0   0.000000   0.000000   0.000000   0.000000   
           TriviaQA   0.0   0.000000   0.000000   0.000000   0.000000   
           MedMCQA    0.0   0.000000   0.000000   0.000000   0.000000   
           MedQA      0.0   0.000000   0.000000   0.000000   0.000000   

                           SAR         LS         SE  Ours(MSIN)  Ours(LSIN)  
Model      Dataset                                                            
Vicuna-7B  SciQ      37.240506  65.970219  29.431648   77.883134   76.292697  
           CoQA      43.827809  60.846751  43.827809   67.980782   62.625648  
           TriviaQA  26.458390  69.045889  26.082792   69.028374   66.081105  
           MedMCQA   46.276803  57.399610  38.456790   69.538012   64.716374  
           MedQA     48.660917  49.752863  48.099036   63.744489   54.267510  
Vicuna-13B SciQ       0.000000   0.000000   0.000000    0.000000    0.000000  
           CoQA       0.000000   0.000000   0.000000    0.000000    0.000000  
           TriviaQA   0.000000   0.000000   0.000000    0.000000    0.000000  
           MedMCQA    0.000000   0.000000   0.000000    0.000000    0.000000  
           MedQA      0.000000   0.000000   0.000000    0.000000    0.000000  
Vicuna-33B SciQ       0.000000   0.000000   0.000000    0.000000    0.000000  
           CoQA       0.000000   0.000000   0.000000    0.000000    0.000000  
           TriviaQA   0.000000   0.000000   0.000000    0.000000    0.000000  
           MedMCQA    0.000000   0.000000   0.000000    0.000000    0.000000  
           MedQA      0.000000   0.000000   0.000000    0.000000    0.000000

Long Prompt Main Result:


ACC         PE      LN-PE   TokenSAR    SentSAR  \
Model      Dataset                                                      
Vicuna-7B  SciQ      66.4  35.535894  47.704210  43.572953  36.383929   
           CoQA      63.2  40.189873  48.161461  48.768575  40.066043   
           TriviaQA  31.0  32.229547  56.373072  57.310893  34.487611   
           MedMCQA   18.2  38.280179  47.716220  46.256616  38.862543   
           MedQA     21.5  36.254777  47.194786  45.922678  36.793364   
Vicuna-13B SciQ       0.0   0.000000   0.000000   0.000000   0.000000   
           CoQA       0.0   0.000000   0.000000   0.000000   0.000000   
           TriviaQA   0.0   0.000000   0.000000   0.000000   0.000000   
           MedMCQA    0.0   0.000000   0.000000   0.000000   0.000000   
           MedQA      0.0   0.000000   0.000000   0.000000   0.000000   
Vicuna-33B SciQ       0.0   0.000000   0.000000   0.000000   0.000000   
           CoQA       0.0   0.000000   0.000000   0.000000   0.000000   
           TriviaQA   0.0   0.000000   0.000000   0.000000   0.000000   
           MedMCQA    0.0   0.000000   0.000000   0.000000   0.000000   
           MedQA      0.0   0.000000   0.000000   0.000000   0.000000   

                           SAR         LS         SE  Ours(MSIN)  Ours(LSIN)  
Model      Dataset                                                            
Vicuna-7B  SciQ      39.945048  61.079810  61.724129   74.628873   76.556001  
           CoQA      48.073748  57.899353  59.589983   72.203495   67.323370  
           TriviaQA  48.880318  71.531089  57.240767   77.067321   77.943198  
           MedMCQA   43.508692  63.210323  60.729063   69.064859   64.900656  
           MedQA     39.849800  62.509850  61.588505   73.482151   68.652348  
Vicuna-13B SciQ       0.000000   0.000000   0.000000    0.000000    0.000000  
           CoQA       0.000000   0.000000   0.000000    0.000000    0.000000  
           TriviaQA   0.000000   0.000000   0.000000    0.000000    0.000000  
           MedMCQA    0.000000   0.000000   0.000000    0.000000    0.000000  
           MedQA      0.000000   0.000000   0.000000    0.000000    0.000000  
Vicuna-33B SciQ       0.000000   0.000000   0.000000    0.000000    0.000000  
           CoQA       0.000000   0.000000   0.000000    0.000000    0.000000  
           TriviaQA   0.000000   0.000000   0.000000    0.000000    0.000000  
           MedMCQA    0.000000   0.000000   0.000000    0.000000    0.000000  
           MedQA      0.000000   0.000000   0.000000    0.000000    0.000000

In [42]:
# Generalization1: Cross Dataset and Cross Prompt Evaluation
model_name = 'vicuna-7b-v1.1'


def plot_cross_dst_matrix(model_name, u_metric, c_metric, c_th):
    fig_index = [(name, type) for type in dst_types for name in dst_names]
    fig_axis = list(map(lambda idx: f"{dst_names_alias[idx[0]]}-{idx[1]}", fig_index))
    cross_eval_matrix = torch.zeros(len(dst_types) * len(dst_names), len(dst_types) * len(dst_names))
    fig = go.Figure()
    annotations = []

    for i, (train_dst_name, train_dst_type) in enumerate(fig_index):
        for j, (test_dst_name, test_dst_type) in enumerate(fig_index):
            result_path = get_eval_cross_result_path(model_name, train_dst_name, train_dst_type, test_dst_name, test_dst_type, u_metric.split("_")[-1])
            if os.path.exists(f"{result_path}/dataset_info.json"):
                cross_eval_result = Dataset.load_from_disk(result_path)
                cross_eval_matrix[j][i] = get_auroc(cross_eval_result, u_metric, c_metric, c_th) * 100
                annotations.append(dict(
                    x=i,
                    y=j,
                    text=f"{cross_eval_matrix[j][i].item():.2f}",
                    showarrow=False,
                    font=dict(
                        color='white'
                    )
                ))
    fig.add_trace(go.Heatmap(z=cross_eval_matrix, x=fig_axis, y=fig_axis, colorscale='Inferno'))
    fig.update_layout(
        title_text=f"Model: {model_names_alias[model_name]} Correctness Metric: {c_metric} Method: {u_metric} Cross Eval Results",
        xaxis_title="Train Dataset",
        yaxis_title="Test Dataset",
        width=1000,
        height=1000,
        annotations=annotations
    )
    fig.show()
    overall_average_drop = cross_eval_matrix.mean().item() - cross_eval_matrix.diag().mean().item()
    short_average_drop = cross_eval_matrix[:5, :5].mean().item() - cross_eval_matrix[:5, :5].diag().mean().item()
    long_average_drop = cross_eval_matrix[5:, 5:].mean().item() - cross_eval_matrix[5:, 5:].diag().mean().item()
    cross_dst_average_drop = (short_average_drop + long_average_drop) / 2
    cross_prompt_average_drop = 0
    for i in range(len(cross_eval_matrix)):
        cross_prompt_average_drop += cross_eval_matrix[i][(i + len(cross_eval_matrix) // 2) % len(cross_eval_matrix)].item()
    cross_prompt_average_drop /= len(cross_eval_matrix)
    print(f"Overall Average Drop: {overall_average_drop:.2f}")
    print(f"Short Average Drop: {short_average_drop:.2f}")
    print(f"Long Average Drop: {long_average_drop:.2f}")
    print(f"Cross Dst Average Drop: {cross_dst_average_drop:.2f}")
    print(f"Cross Prompt Average Drop: {cross_prompt_average_drop:.2f}")
    for i in range(len(cross_eval_matrix)):
        dst_drop = cross_eval_matrix[:, i].mean().item() - cross_eval_matrix[i, i].item()
        print(f"Train Dst:{fig_axis[i]} Average Drop: {dst_drop:.2f}")


plot_cross_dst_matrix(model_name, u_metric='u_score_ours_mean_soft_rougel', c_metric='include', c_th=0.3)
plot_cross_dst_matrix(model_name, u_metric='u_score_ours_mean_soft_include', c_metric='include', c_th=0.3)

Overall Average Drop: -3.00
Short Average Drop: -3.00
Long Average Drop: -1.92
Cross Dst Average Drop: -2.46
Cross Prompt Average Drop: 61.09
Train Dst:SciQ-short Average Drop: -11.04
Train Dst:CoQA-short Average Drop: -0.18
Train Dst:TriviaQA-short Average Drop: -6.29
Train Dst:MedMCQA-short Average Drop: -1.85
Train Dst:MedQA-short Average Drop: 12.84
Train Dst:SciQ-long Average Drop: -5.55
Train Dst:CoQA-long Average Drop: -1.98
Train Dst:TriviaQA-long Average Drop: -9.89
Train Dst:MedMCQA-long Average Drop: 0.60
Train Dst:MedQA-long Average Drop: -6.64


Overall Average Drop: -9.46
Short Average Drop: -7.51
Long Average Drop: -7.03
Cross Dst Average Drop: -7.27
Cross Prompt Average Drop: 63.75
Train Dst:SciQ-short Average Drop: -13.57
Train Dst:CoQA-short Average Drop: -3.15
Train Dst:TriviaQA-short Average Drop: -8.81
Train Dst:MedMCQA-short Average Drop: -10.06
Train Dst:MedQA-short Average Drop: -7.27
Train Dst:SciQ-long Average Drop: -8.70
Train Dst:CoQA-long Average Drop: -8.61
Train Dst:TriviaQA-long Average Drop: -17.31
Train Dst:MedMCQA-long Average Drop: -5.82
Train Dst:MedQA-long Average Drop: -11.26


In [52]:
# Generalization2: Cross Correctness Metric Evaluation
model_name = 'vicuna-7b-v1.1'
c_th = 0.3

c_metric_index = [name for name in c_metrics for i in range(len(dst_names))]
print(c_metric_index)
dst_names_index = [dst_names_alias[name] for name in dst_names] * len(c_metrics)
print(dst_names_index)
multi_index = pd.MultiIndex.from_tuples(zip(c_metric_index, dst_names_index), names=['Correctness Metric', 'Dataset'])
columns=['ACC']+[u for u in u_metric_alias.values() if 'Ours' in u]
print(columns)
cross_c_metric_results_short = pd.DataFrame(columns=columns, index=multi_index).astype(float)
cross_c_metric_results_long = deepcopy(cross_c_metric_results_short)

for dst_type in dst_types:
    result = cross_c_metric_results_short if dst_type == 'short' else cross_c_metric_results_long
    for c_metric in c_metrics:
        for dst_name in dst_names:
            new_row = {k: 0. for k in columns}
            result_path = get_eval_main_result_path(model_name, dst_name, dst_type)
            if os.path.exists(result_path):
                test_dst = Dataset.load_from_disk(result_path)
                new_row['ACC'] = get_acc_by_c_th(test_dst, c_metric, c_th) * 100
                for u_metric, u_metric_name in u_metric_alias.items():
                    if u_metric_name in columns and u_metric in test_dst.column_names:
                        new_row[u_metric_name] = get_auroc(test_dst, u_metric, c_metric, c_th) * 100
            result.loc[(c_metric, dst_names_alias[dst_name])] = new_row

print("Short Prompt Cross Correctness Metric Result:")
display(cross_c_metric_results_short)

print("Long Prompt Cross Correctness Metric Result:")
display(cross_c_metric_results_long)


['rougel', 'rougel', 'rougel', 'rougel', 'rougel', 'sentsim', 'sentsim', 'sentsim', 'sentsim', 'sentsim', 'include', 'include', 'include', 'include', 'include']
['SciQ', 'CoQA', 'TriviaQA', 'MedMCQA', 'MedQA', 'SciQ', 'CoQA', 'TriviaQA', 'MedMCQA', 'MedQA', 'SciQ', 'CoQA', 'TriviaQA', 'MedMCQA', 'MedQA']
['ACC', 'Ours(MSRL)', 'Ours(LSRL)', 'Ours(MSIN)', 'Ours(LSIN)', 'Ours(MSSI)', 'Ours(LSSI)']
Short Prompt Cross Correctness Metric Result:


ACC  Ours(MSRL)  Ours(LSRL)  Ours(MSIN)  \
Correctness Metric Dataset                                              
rougel             SciQ      66.6   86.126171   83.728714   82.099540   
                   CoQA      52.2   83.330929   81.749467   74.311066   
                   TriviaQA  52.9   72.515743   70.878836   67.235781   
                   MedMCQA   25.1   88.772547   85.187687   68.545311   
                   MedQA      4.9   92.167214   76.766669   52.975386   
sentsim            SciQ      93.8   75.540787   71.948724   69.774400   
                   CoQA      82.6   80.527956   80.657371   68.230776   
                   TriviaQA  83.1   76.876081   77.903574   73.306916   
                   MedMCQA   76.6   59.919997   56.319319   45.250887   
                   MedQA     72.4   62.827788   50.571253   41.695442   
include            SciQ      64.9   75.738041   72.440178   77.883134   
                   CoQA      68.2   61.439717   60.035228   67.980782   
                   TriviaQA  36.3   68.979722   69.297369   69.028374   
                   MedMCQA   19.0   62.399935   65.441520   69.538012   
                   MedQA     22.8   44.922621   50.461038   63.744489   

                             Ours(LSIN)  Ours(MSSI)  Ours(LSSI)  
Correctness Metric Dataset                                       
rougel             SciQ       77.029050         0.0         0.0  
                   CoQA       74.858526         0.0         0.0  
                   TriviaQA   61.874546         0.0         0.0  
                   MedMCQA    74.425396         0.0         0.0  
                   MedQA      59.774888         0.0         0.0  
sentsim            SciQ       66.438545         0.0         0.0  
                   CoQA       67.227464         0.0         0.0  
                   TriviaQA   69.411275         0.0         0.0  
                   MedMCQA    51.403115         0.0         0.0  
                   MedQA      50.636810         0.0         0.0  
include            SciQ       76.292697         0.0         0.0  
                   CoQA       62.625648         0.0         0.0  
                   TriviaQA   66.081105         0.0         0.0  
                   MedMCQA    64.716374         0.0         0.0  
                   MedQA      54.267510         0.0         0.0

Long Prompt Cross Correctness Metric Result:


ACC  Ours(MSRL)  Ours(LSRL)  Ours(MSIN)  \
Correctness Metric Dataset                                              
rougel             SciQ      20.8   88.927739   86.360905   61.166958   
                   CoQA      38.6   78.890652   72.618184   66.377783   
                   TriviaQA  36.9   85.726833   87.194156   81.725785   
                   MedMCQA   16.5   80.601343   75.462529   58.014154   
                   MedQA     15.0   79.352549   76.653333   71.553333   
sentsim            SciQ      83.1   73.070515   73.907889   76.483740   
                   CoQA      77.4   78.969152   72.891084   66.300793   
                   TriviaQA  63.3   88.677032   90.737632   89.708408   
                   MedMCQA   54.4   65.451658   62.038426   57.534951   
                   MedQA     56.5   68.231919   65.489777   68.148713   
include            SciQ      66.4   67.327569   67.957769   74.628873   
                   CoQA      63.2   62.064873   63.187947   72.203495   
                   TriviaQA  31.0   69.380552   73.266947   77.067321   
                   MedMCQA   18.2   59.449475   57.264435   69.064859   
                   MedQA     21.5   66.795438   62.590727   73.482151   

                             Ours(LSIN)  Ours(MSSI)  Ours(LSSI)  
Correctness Metric Dataset                                       
rougel             SciQ       67.436990         0.0         0.0  
                   CoQA       67.801387         0.0         0.0  
                   TriviaQA   76.094426         0.0         0.0  
                   MedMCQA    60.108510         0.0         0.0  
                   MedQA      66.163137         0.0         0.0  
sentsim            SciQ       78.768006         0.0         0.0  
                   CoQA       70.418010         0.0         0.0  
                   TriviaQA   86.752026         0.0         0.0  
                   MedMCQA    58.685863         0.0         0.0  
                   MedQA      64.616824         0.0         0.0  
include            SciQ       76.556001         0.0         0.0  
                   CoQA       67.323370         0.0         0.0  
                   TriviaQA   77.943198         0.0         0.0  
                   MedMCQA    64.900656         0.0         0.0  
                   MedQA      68.652348         0.0         0.0

In [ ]:
# Efficiency: Get Efficiency Results
model_name = 'vicuna-7b-v1.1'
dst_name = 'sciq'
dst_type = 'long'


def plot_efficiency_results(model_name, dst_name, dst_type):
    fig = go.Figure()
    result_path = get_eval_main_result_path(model_name, dst_name, dst_type)

In [ ]:
# Ablation Study: Get Ablation Results

In [ ]:
# Sensitivity Analysis : Get Sensitivity Results

In [ ]:
# Case Study: show token level u_score
example = test_dst.filter(lambda x: x['rougel'] < 0.1)[1]
example = test_dst[2]
print(f"gt:{example['gt']}")
print(f"options:{example['options']}")

str_tokens = model.to_str_tokens(f":{example['washed_answer']}", prepend_bos=False)[1:]
fig = make_subplots(rows=2, cols=1, subplot_titles=("Token Level", "Sentence Level"), row_heights=[0.5, 0.5])

fig.add_trace(go.Scatter(x=list(range(len(str_tokens))), y=example['u_score_pe_all'], mode='lines+markers'), row=1, col=1)
fig.update_xaxes(title_text='Token', tickvals=list(range(len(str_tokens))), ticktext=str_tokens, row=1, col=1)

sentence_u_score_pe_all = []
indices = [0] + [i for i, x in enumerate(str_tokens) if x == '.'] + [-1]
spans = [(indices[i], indices[i + 1]) for i in range(len(indices) - 1)]
print(len(indices))
for span in spans:
    sentence_score = sum(example['u_score_pe_all'][span[0]:span[1]]) / (span[1] - span[0])
    sentence_u_score_pe_all.extend([sentence_score] * (span[1] - span[0]))
sentence_u_score_pe_all.append(sentence_u_score_pe_all[-1])
# print(str_tokens)
for i, sentence in enumerate(example['washed_answer'].split(".")):
    print(i + 1, sentence.replace("\n", ' ').strip())
# print(len(example['u_score_pe_all']))
# print(sentence_u_score_pe_all)
# print(len(sentence_u_score_pe_all))

fig.add_trace(go.Scatter(x=list(range(len(str_tokens))), y=sentence_u_score_pe_all, mode='lines+markers'), row=2, col=1)
fig.update_xaxes(title_text='Sentence', tickvals=list(range(len(str_tokens))), ticktext=str_tokens, row=2, col=1)

fig.update_layout(height=1000, width=2500, margin=dict(l=0, r=0, b=50, t=50), title_text=example['washed_answer'])
fig.show()